In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

from  bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 
import time

import warnings
warnings.filterwarnings('ignore')

import string, nltk, re
from konlpy.tag import Okt
from wordcloud import WordCloud

from PIL import Image
from wordcloud import ImageColorGenerator

import os
driver_name =  os.environ['HOMEDRIVE'] + os.environ['HOMEPATH'] + '\\Downloads\\chromedriver_win32\\chromedriver.exe'

### 영화관입장권통합전산망 KOBIS 에서 영화 정보 크롤링

In [17]:
url = "https://www.kobis.or.kr/kobis/business/mast/mvie/searchMovieList.do"

home_url = "https://www.kobis.or.kr/"

In [47]:
driver = webdriver.Chrome(driver_name)
driver.get(url)
time.sleep(2)

TOTAL_COUNT = 50
START_COUNT = 0
SUB_COUNT = 0
for i in range(START_COUNT):
    driver.find_element(By.CSS_SELECTOR, '.paging > a.btn.next').click()
    time.sleep(3)
    
lis = driver.find_elements(By.CSS_SELECTOR, '.paging > ul > li')
current = 0
for k in range(SUB_COUNT):
    current += 1
    lis[current].click()
    time.sleep(3)
    lis = driver.find_elements(By.CSS_SELECTOR, '.paging > ul > li')

lis = driver.find_elements(By.CSS_SELECTOR, '.paging > ul > li')

In [54]:

driver = webdriver.Chrome(driver_name)

driver.get(url)
time.sleep(2)
ORDER = '03'
n_page = 0
# paging
lis = driver.find_elements(By.CSS_SELECTOR, '.paging > ul > li')
# print(lis)

end, current = len(lis), 0
lines = []

while True:

    try:
        trs = driver.find_elements(By.CSS_SELECTOR, 'tbody > tr')
    except:
        print('nPgae = ', n_page)
        break

    for tr in trs:

        td = tr.find_elements(By.TAG_NAME, 'td')

        kind = td[5].text
        genre = td[6].text
        
        if kind =='' or kind =='단편' or kind == "기타" or kind == '옴니버스':
            continue
        if '성인' in genre or genre == '' :
            continue

        td[0].find_element(By.TAG_NAME, 'a').click()
        time.sleep(2)

        # 영화 제목
        title = driver.find_element(By.CSS_SELECTOR, '.hd_layer > div').text
        find_index = title.find(')') 
        if find_index >= 0 :
            title = title[:find_index+1]
        
        infos = driver.find_element(By.CSS_SELECTOR, 'div.basic')

        # 시놉시스가 없으면 pass
        try :
            synopsis = infos.find_element(By.CSS_SELECTOR, '.info.info2 > p').text
        except :
            driver.find_element(By.CSS_SELECTOR, '.hd_layer > a.close').click()
            continue

        infos_head = infos.find_element(By.CSS_SELECTOR, '.ovf.info.info1')
            
        # 이미지 URL
        img = infos_head.find_element(By.CSS_SELECTOR, 'a > img').get_attribute('src')

        # 요약 정보
        dds = infos_head.find_elements(By.CSS_SELECTOR, 'dl > dd')

        # 코드 정보
        code = dds[0].text
        summering = dds[3].text
        # 개봉일
        first_day = dds[-6].text
        # 제작년도
        production_year = dds[-5].text
    
        
        # 감독 / 주연 배우
        tmp = infos.find_elements(By.CSS_SELECTOR, '.info.info2 > .staffMore > dl > div')
        movie_director, star_actor = '', ''
        find_count = 0
        for tp in tmp:
            if tp.text.find('감독') >= 0:
                movie_director = tp.text.replace('감독', '').strip()
                find_count += 1
                
            if tp.text.find('[주연]') >=0:
                star_actor = tp.text[tp.text.find('[주연]') + 4 : ].strip()
                print(star_actor)
                find_count += 1

            if find_count == 2: break

        lines.append({'code': code, 'title': title, 'genre': genre, 'summering': summering, 
                    'first_day': first_day, 'production_year': production_year, 
                    'movie_director':movie_director, 'star_actor': star_actor, 
                    'img': img, 'synopsis': synopsis})

        driver.find_element(By.CSS_SELECTOR, '.hd_layer > a.close').click()
        time.sleep(2)


    current += 1
    n_page += 1

    if n_page == TOTAL_COUNT: break

    if end == current:
        try:
            driver.find_element(By.CSS_SELECTOR, '.paging > a.btn.next').click()
            time.sleep(3)
            lis = driver.find_elements(By.CSS_SELECTOR, '.paging > ul > li')
            end, current = len(lis), 0
        except:
            break
    else:
        # 다음 페이지 넘어가기
        lis = driver.find_elements(By.CSS_SELECTOR, '.paging > ul > li')
        # print(lis)
        lis[current].click()
        time.sleep(3)

df = pd.DataFrame(lines)

df.to_csv(f'../static/data/test{ORDER}.csv', index=False)

조경이(핑크퐁) | 이소은(호기) | 박요한(진저브레드맨) | 김새해(제니) | 장예나(아기상어)
하정우 | 주지훈
줄리엣 비노쉬 | 욜랭드 모로 | 노에미 르보브스키 | 에두아르 바에르 | 프랑수아 베를레앙
김재화 | 조민재 | 박강섭
신은수


In [38]:
len(lines)

1226

In [39]:
df = pd.DataFrame(lines)


In [34]:
driver.close()

In [ ]:
df.tail()

In [41]:
df.to_csv(f'../static/data/test{ORDER}.csv', index=False)